In [ ]:
#CÓDIGO CORRESPONDIENTE AL EXÁMEN #1.
#PRESENTADO POR: Andrés Theran y Daniel Ortíz
#UNIVERSIDAD DE CÓRDOBA

#Paquete  necesario para el código: 
# pip install notebook scikit-learn numpy

#PARTE INICIAL PARA OBTENER INFORMACIÓN DEL CONJUNTO DE DATOS.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Cargar el dataset
california = fetch_california_housing()
X = california.data
y = california.target.reshape(-1, 1) #Para manejar los valores objetivos como un vector columna.
feature_names = california.feature_names
print(X.shape)
print(y.shape)
print('Descripción del dataset:')
print(california.DESCR[:1500])


In [ ]:
#Preprocesamiento : normalización de datos y escogencia de entrenamiento (80%) y prueba (20%)

#1. Escogemos conjunto de training y conjunto de test.
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

#2. Normalizamos los datos de training y los de test (teniendo en cuenta solo la información estadística del training).
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

In [ ]:

#Definimos los parámetros.

def inicializar_parametros(n_caracteristicas):
    
    # Pesos aleatorios pequeños
    w = np.random.randn(n_caracteristicas, 1) * 0.01
        
    # Sesgo en cero
    b = 0.0
    
    return w, b



In [ ]:
#Definimos la suma ponderada de las caracteristicas con sus pesos para los ejemplos
def propagacion_adelante(X, w, b):
    z = X @ w + b 
    return z

In [ ]:
#Ejemplo de predicción para 5 registros

#asignar caracteristicas
n_caracteristicas = X_train_scaled.shape[1]

#asignar pesos y sesgos.
w, b = inicializar_parametros(n_caracteristicas)

X_train_exa = X_train_scaled[:5]
y_real_exa = y_train[:5]

# Predicciones iniciales
y_pred_exa = propagacion_adelante(X_train_exa, w, b)

print("Predicciones iniciales:")
print(y_pred_exa)

print("\nValores reales:")
print(y_real_exa)


In [ ]:

#Función que calcula la pérdida para este caso de regresión

def calcular_perdida(y_pred, y_real):
    
    m = y_real.shape[0]
    
    mse = (1/m) * np.sum((y_pred - y_real)**2)
    
    return mse


In [ ]:
#Definimos la predicción de las viviendas para los parámetros iniciales. (no se ha entrado la red aún)
y_pred_t=propagacion_adelante(X_train_scaled,w,b)

#Calculamos la pérdida para esta primera predicción
mse_i=calcular_perdida(y_pred_t,y_train)
print(mse_i)

In [ ]:
#Definimos los gradientes con respecto a los parámetros

def calcular_gradientes (X, y_pred,y_real):
    m=X.shape[0]

    error=y_pred-y_real
    
    dw=(2/m)*(X.T @ error)
    db=(2/m)*np.sum(error)

    return dw,db


In [ ]:
#Definimos la actualización de parámetros.
def actualizar_parametros (w,b,dw,db,learning_rate):
    
    w= w - learning_rate*dw 
    b= b - learning_rate*db    

    return w,b



In [ ]:
# Función de entrenamiento del perceptrón para regresión.
def entrenar_perceptron(X_train, y_train, learning_rate, epochs):
    n_caracteristicas = X_train_scaled.shape[1]

#asignar pesos y sesgos.
    w, b = inicializar_parametros(n_caracteristicas)
    historial_perdida = []
    
    for epoca in range(epochs):
        
        # 1️ Propagación adelante
        y_pred = propagacion_adelante(X_train, w, b)
        
        # 2️ Calcular pérdida
        perdida = calcular_perdida(y_pred, y_train)
        historial_perdida.append(perdida)
        
        # 3️ Calcular gradientes
        dw, db = calcular_gradientes(X_train, y_pred, y_train)
        
        # 4️ Actualizar parámetros
        w, b = actualizar_parametros(w, b, dw, db, learning_rate)
        
        if epoca % 100 == 0:
            print(f'Época {epoca}, Pérdida: {perdida:.4f}')
            print("-"*50)
    return w, b, historial_perdida





In [ ]:


#Entrenamos el perceptrón con diferentes tasas de aprendizaje y guardamos los pesos, sesgos e historial de pérdida para cada caso.
h4=w4,b4,historial_perdida4=entrenar_perceptron(X_train_scaled,y_train,1.0,1000)
h1=w1,b1,historial_perdida1=entrenar_perceptron(X_train_scaled, y_train,0.01,1000)
h2=w2,b2,historial_perdida2=entrenar_perceptron(X_train_scaled,y_train,0.1,1000)
h3=w3,b3,historial_perdida3=entrenar_perceptron(X_train_scaled,y_train,0.001,1000)
print("Guardamos los pesos, sesgos e historial de pérdida para cada tasa de aprendizaje.")

In [ ]:

#Visualizamos la evolución de la pérdida para la tasa de aprendizaje 0.01
plt.plot(historial_perdida1)
plt.xlabel("Épocas")
plt.ylabel("MSE")
plt.title("Evolución de la pérdida")
plt.show()


In [ ]:
#Comparación de la evolución de la pérdida para las diferentes tasas de aprendizaje.
plt.figure()
plt.plot(historial_perdida4, label="lr=1.0")
plt.plot(historial_perdida3, label="lr=0.001")
plt.plot(historial_perdida1, label="lr=0.01")
plt.plot(historial_perdida2, label="lr=0.1")

plt.xlabel("Época")
plt.ylabel("MSE")
plt.ylim(0,6)
plt.title("Comparación de tasas de aprendizaje")
plt.legend()

plt.show()



In [ ]:
#Obtenemos el MSE final de entrenamiento para la tasa de aprendizaje 0.01
mse_entrenamiento_final = historial_perdida1[-1]
print("MSE final de entrenamiento:", mse_entrenamiento_final)


In [ ]:
#Obtenemos el MSE en el conjunto de prueba para el conjunto de parámetros entrenados con tasa de aprendizaje 0.01
y_test_pred = propagacion_adelante(X_test_scaled, w1, b1)
mse_test = calcular_perdida(y_test_pred, y_test)
print("MSE en conjunto de prueba:", mse_test)

In [ ]:
#Visualizamos las predicciones vs los valores reales para el conjunto de prueba.
# la  línea roja discontinua representa la situación ideal donde las predicciones coinciden exactamente con los valores reales (y=x).

plt.figure(figsize=(7,7))
plt.scatter(y_test, y_test_pred, s=10,alpha=0.6)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--', label='y=x')
plt.xlabel("Valores reales (y_test)")
plt.ylabel("Predicciones")
plt.title("Predicciones vs valores reales")
plt.legend()
plt.show()


In [ ]:
#Calculamos el coeficiente de determinación R² para evaluar la calidad de las predicciones.
SS_res = np.sum((y_test - y_test_pred)**2)
SS_tot = np.sum((y_test - np.mean(y_test))**2)

R2 = 1 - (SS_res / SS_tot)

print("R²:", R2)